# 🚀 LangGraph Avanzado - Multi-Modelo y Arquitecturas Complejas
## Clase 6: Llevando LangGraph al Siguiente Nivel

### 🎯 Objetivos de esta clase:

1. **Usar diferentes modelos en diferentes nodos** según la complejidad de la tarea
2. **Integrar Ollama con Docker** para modelos locales
3. **Enrutamiento inteligente** entre modelos (GPT-4 para tareas complejas, GPT-3.5 para simples)
4. **Persistencia avanzada** con checkpoints
5. **Arquitecturas de agentes** real-world

### 💡 Caso de uso real:

Imagine una empresa de construcción donde:
- **Preguntas simples** (stock, horarios) → Modelo rápido/barato (GPT-3.5 o Llama local)
- **Análisis técnicos** (especificaciones, comparaciones) → Modelo potente (GPT-4)
- **Cálculos y validaciones** → Modelo local sin costo (Llama)
- **Respuestas finales** → Modelo premium para calidad

**Ahorro de costos:** Usar GPT-4 solo cuando realmente se necesita puede reducir costos hasta 90%

---

## 📦 PARTE 0: Instalación y Setup

### Paquetes necesarios

In [13]:
!pip install langgraph langchain-openai langchain-ollama langchain-core langchain-community python-dotenv -q

print("✅ Paquetes instalados")

✅ Paquetes instalados


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv('OPENAI_API_KEY'):
    raise ValueError("⚠️ OPENAI_API_KEY no encontrada")

print("✅ Configuración lista")
print("\n📋 Requisitos:")
print("  1. OpenAI API Key configurada")
print("  2. Docker instalado (para modelos locales)")
print("  3. Ollama corriendo en http://localhost:11434")

✅ Configuración lista

📋 Requisitos:
  1. OpenAI API Key configurada
  2. Docker instalado (para modelos locales)
  3. Ollama corriendo en http://localhost:11434


---
# 🎯 PARTE 1: Múltiples Modelos en un Grafo

## El Problema:

En un sistema real, NO todas las tareas requieren el mismo modelo:

```
Tarea Simple ("¿Horario?") → GPT-3.5 Turbo ($0.0005/1K tokens) ✅
                             vs
                             GPT-4 ($0.03/1K tokens)        ❌ (60x más caro)

Tarea Compleja ("Analizar especificaciones técnicas") → GPT-4 ✅
```

## La Solución: Router Inteligente

Crearemos un grafo que:
1. **Clasifica** la complejidad de la pregunta
2. **Enruta** a diferentes modelos según complejidad
3. **Optimiza** costos automáticamente

## 1.1 Configurar Múltiples Modelos

In [3]:
from langchain_openai import ChatOpenAI
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, SystemMessage

# 📌 MÚLTIPLES MODELOS PARA DIFERENTES TAREAS

# 1. Modelo BARATO Y RÁPIDO para clasificación inicial
llm_classifier = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    model_kwargs={"seed": 42}  # Para reproducibilidad
)

# 2. Modelo ECONÓMICO para tareas simples
llm_simple = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.3
)

# 3. Modelo POTENTE para tareas complejas
llm_advanced = ChatOpenAI(
    model="gpt-4o-mini",  # GPT-4 Turbo es más barato que GPT-4 original
    temperature=0.5
)

# 4. Modelo PREMIUM para respuestas finales críticas
llm_premium = ChatOpenAI(
    model="gpt-4o",
    temperature=0.7
)

print("✅ Modelos configurados:")
print("\n   🔵 Classifier: gpt-3.5-turbo (rápido y barato)")
print("   🟢 Simple: gpt-3.5-turbo (tareas básicas)")
print("   🟡 Advanced: gpt-4o-mini (análisis complejos)")
print("   🔴 Premium: gpt-4o (respuestas críticas)")

print("\n💰 Comparación de costos (por 1M tokens input):")
print("   gpt-3.5-turbo: $0.50")
print("   gpt-4o-mini:   $0.15")
print("   gpt-4o:        $2.50")
print("\n   Ahorro potencial: Hasta 5x usando routing inteligente")

e:\Programs\Anaconda\envs\curos-ia\Lib\site-packages\IPython\core\interactiveshell.py:3517: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


✅ Modelos configurados:

   🔵 Classifier: gpt-3.5-turbo (rápido y barato)
   🟢 Simple: gpt-3.5-turbo (tareas básicas)
   🟡 Advanced: gpt-4o-mini (análisis complejos)
   🔴 Premium: gpt-4o (respuestas críticas)

💰 Comparación de costos (por 1M tokens input):
   gpt-3.5-turbo: $0.50
   gpt-4o-mini:   $0.15
   gpt-4o:        $2.50

   Ahorro potencial: Hasta 5x usando routing inteligente


## 1.2 Definir el State y Nodos

In [4]:
# State que pasa entre nodos
class StateMultiModelo(TypedDict):
    pregunta: str                    # Pregunta del usuario
    complejidad: str                 # "simple", "complejo", "critico"
    contexto_tecnico: str            # Información técnica extraída
    respuesta: str                   # Respuesta final
    modelo_usado: str                # Qué modelo se usó
    tokens_estimados: int            # Estimación de tokens usados
    requiere_validacion: bool        # Si necesita validación humana

print("✅ State definido con 7 campos para tracking completo")

✅ State definido con 7 campos para tracking completo


In [ ]:
# 🎯 NODO 1: Clasificador de Complejidad
def clasificar_pregunta(state: StateMultiModelo) -> StateMultiModelo:
    """
    Usa GPT-3.5 (barato) para clasificar la complejidad de la pregunta.
    Esto determina qué modelo usaremos después.
    """
    print(f"\n🔍 [CLASSIFIER] Analizando: '{state['pregunta']}'")
    
    prompt = f"""Clasifica la complejidad de esta pregunta en una de estas categorías:

SIMPLE: Preguntas básicas sobre horarios, ubicación, contacto, stock simple.
Ejemplos: "¿A qué hora abren?", "¿Tienen rotomartillos?", "¿Dónde están ubicados?"

COMPLEJO: Requiere análisis técnico, comparaciones, cálculos, recomendaciones.
Ejemplos: "¿Cuál equipo es mejor para demoler concreto armado?", "Necesito comparar especificaciones"

CRITICO: Decisiones importantes, cotizaciones grandes, consultas legales/contractuales.
Ejemplos: "Necesito cotización para proyecto de HNL 100K", "¿Qué garantías ofrecen?"

Pregunta: {state['pregunta']}

Responde SOLO con: SIMPLE, COMPLEJO o CRITICO"""
    
    response = llm_classifier.invoke([HumanMessage(content=prompt)])
    complejidad = response.content.strip().upper()
    
    print(f"   Complejidad detectada: {complejidad}")
    
    return {
        **state,
        "complejidad": complejidad,
        "tokens_estimados": len(state['pregunta']) // 4  # Estimación rough
    }

# 🟢 NODO 2: Responder Simple (GPT-3.5)
def responder_simple(state: StateMultiModelo) -> StateMultiModelo:
    """
    Usa GPT-3.5 para preguntas simples.
    Rápido, barato, suficiente para consultas básicas.
    """
    print("\n🟢 [SIMPLE MODEL] Procesando con GPT-3.5...")
    
    prompt = f"""Eres un asistente de CONCESA, empresa de renta de equipos de construcción.
Responde esta pregunta de forma breve y profesional.

Pregunta: {state['pregunta']}

Respuesta:"""
    
    response = llm_simple.invoke([HumanMessage(content=prompt)])
    
    print(f"   ✅ Respuesta generada ({len(response.content)} chars)")
    
    return {
        **state,
        "respuesta": response.content,
        "modelo_usado": "gpt-3.5-turbo",
        "requiere_validacion": False
    }

# 🟡 NODO 3: Responder Complejo (GPT-4o-mini)
def responder_complejo(state: StateMultiModelo) -> StateMultiModelo:
    """
    Usa GPT-4o-mini para análisis técnicos y comparaciones.
    Más potente que GPT-3.5, ideal para razonamiento.
    """
    print("\n🟡 [ADVANCED MODEL] Procesando con GPT-4o-mini...")
    
    prompt = f"""Eres un experto técnico en equipos de construcción de CONCESA.
Proporciona un análisis detallado y profesional.

Pregunta: {state['pregunta']}

Incluye:
1. Análisis técnico
2. Recomendaciones específicas
3. Consideraciones importantes

Respuesta:"""
    
    response = llm_advanced.invoke([HumanMessage(content=prompt)])
    
    print(f"   ✅ Análisis completo generado ({len(response.content)} chars)")
    
    return {
        **state,
        "respuesta": response.content,
        "modelo_usado": "gpt-4o-mini",
        "requiere_validacion": False
    }

# 🔴 NODO 4: Responder Crítico (GPT-4o)
def responder_critico(state: StateMultiModelo) -> StateMultiModelo:
    """
    Usa GPT-4o para consultas críticas que requieren máxima calidad.
    Más caro pero necesario para decisiones importantes.
    """
    print("\n🔴 [PREMIUM MODEL] Procesando con GPT-4o (modelo premium)...")
    
    prompt = f"""Eres un asesor senior de CONCESA con 20 años de experiencia.
Esta es una consulta CRÍTICA que requiere tu máxima atención y expertise.

Pregunta: {state['pregunta']}

Proporciona:
1. Análisis exhaustivo
2. Recomendaciones basadas en mejores prácticas
3. Consideraciones de riesgo
4. Próximos pasos sugeridos
5. Información de contacto para seguimiento

Esta respuesta será revisada por un humano antes de enviarla.

Respuesta:"""
    
    response = llm_premium.invoke([HumanMessage(content=prompt)])
    
    print(f"   ✅ Análisis premium generado ({len(response.content)} chars)")
    print("   ⚠️ Requiere validación humana")
    
    return {
        **state,
        "respuesta": response.content,
        "modelo_usado": "gpt-4o",
        "requiere_validacion": True  # Casos críticos siempre van a revisión
    }

print("✅ 4 nodos definidos, cada uno con su modelo especializado")

✅ 4 nodos definidos, cada uno con su modelo especializado


## 1.3 Función de Routing (Enrutamiento)

In [6]:
# 🔀 Función que decide a qué nodo ir según la complejidad
def decidir_modelo(state: StateMultiModelo) -> Literal["simple", "complejo", "critico"]:
    """
    Routing inteligente basado en la complejidad detectada.
    Esta función determina el flujo del grafo.
    """
    complejidad = state["complejidad"]
    
    print(f"\n🔀 [ROUTER] Decisión: '{complejidad}' →", end=" ")
    
    if complejidad == "SIMPLE":
        print("Modelo Simple (GPT-3.5)")
        return "simple"
    elif complejidad == "COMPLEJO":
        print("Modelo Avanzado (GPT-4o-mini)")
        return "complejo"
    else:  # CRITICO
        print("Modelo Premium (GPT-4o)")
        return "critico"

print("✅ Router configurado")

✅ Router configurado


## 1.4 Construir el Grafo Multi-Modelo

In [7]:
# 🕸️ Construir el grafo
workflow_multi = StateGraph(StateMultiModelo)

# Agregar nodos
workflow_multi.add_node("clasificar", clasificar_pregunta)
workflow_multi.add_node("simple", responder_simple)
workflow_multi.add_node("complejo", responder_complejo)
workflow_multi.add_node("critico", responder_critico)

# Flujo: START → clasificar → [routing condicional] → END
workflow_multi.add_edge(START, "clasificar")

# Edge condicional desde clasificar
workflow_multi.add_conditional_edges(
    "clasificar",
    decidir_modelo,
    {
        "simple": "simple",
        "complejo": "complejo",
        "critico": "critico"
    }
)

# Todos los modelos terminan
workflow_multi.add_edge("simple", END)
workflow_multi.add_edge("complejo", END)
workflow_multi.add_edge("critico", END)

# Compilar
app_multi = workflow_multi.compile()

print("\n✅ Grafo Multi-Modelo compilado")
print("\n🕸️ Arquitectura:")
print("")
print("        START")
print("          │")
print("          ▼")
print("    ┌─────────────┐")
print("    │ CLASIFICAR  │ (GPT-3.5)")
print("    └─────────────┘")
print("          │")
print("    ¿Complejidad?")
print("     ╱    │    ╲")
print("    ╱     │     ╲")
print(" SIMPLE COMPLEJO CRITICO")
print("   │      │       │")
print("  3.5   4o-mini  4o")
print("   │      │       │")
print("   └──────┴───────┘")
print("          │")
print("         END")


✅ Grafo Multi-Modelo compilado

🕸️ Arquitectura:

        START
          │
          ▼
    ┌─────────────┐
    │ CLASIFICAR  │ (GPT-3.5)
    └─────────────┘
          │
    ¿Complejidad?
     ╱    │    ╲
    ╱     │     ╲
 SIMPLE COMPLEJO CRITICO
   │      │       │
  3.5   4o-mini  4o
   │      │       │
   └──────┴───────┘
          │
         END


## 1.5 🎬 DEMO: Probar el Sistema Multi-Modelo

In [14]:
print("="*80)
print("🎬 DEMO: SISTEMA MULTI-MODELO EN ACCIÓN")
print("="*80)

# Preparar preguntas de diferentes complejidades
preguntas_test = [
    {
        "pregunta": "¿A qué hora abren?",
        "esperado": "SIMPLE"
    },
    {
        "pregunta": "¿Qué equipo de demolición me recomiendan para un edificio de 5 pisos con concreto armado?",
        "esperado": "COMPLEJO"
    },
    {
        "pregunta": "Necesito cotización para un proyecto de construcción de 6 meses con múltiples equipos valorado en 150,000. ¿Qué garantías y términos contractuales ofrecen?",
        "esperado": "CRITICO"
    }
]

for i, test in enumerate(preguntas_test, 1):
    print(f"\n{'='*80}")
    print(f"PRUEBA #{i} - Se espera complejidad: {test['esperado']}")
    print(f"{'='*80}")
    print(f"\n👤 Pregunta: {test['pregunta']}")
    
    # Ejecutar el grafo
    resultado = app_multi.invoke({
        "pregunta": test['pregunta'],
        "complejidad": "",
        "contexto_tecnico": "",
        "respuesta": "",
        "modelo_usado": "",
        "tokens_estimados": 0,
        "requiere_validacion": False
    })
    
    # Mostrar resultados
    print(f"\n{'─'*80}")
    print("📊 RESULTADO:")
    print(f"{'─'*80}")
    print(f"\n🤖 Modelo usado: {resultado['modelo_usado']}")
    print(f"📏 Complejidad detectada: {resultado['complejidad']}")
    print(f"⚠️ Requiere validación humana: {'Sí' if resultado['requiere_validacion'] else 'No'}")
    print(f"\n💬 Respuesta:\n")
    print(resultado['respuesta'])

print(f"\n\n{'='*80}")
print("✅ DEMO COMPLETADA")
print("="*80)
print("\n💡 Observa cómo el sistema:")
print("   1. Clasificó automáticamente cada pregunta")
print("   2. Usó el modelo apropiado para cada caso")
print("   3. Marcó las respuestas críticas para revisión humana")
print("   4. Optimizó costos usando modelos más baratos cuando es suficiente")

🎬 DEMO: SISTEMA MULTI-MODELO EN ACCIÓN

PRUEBA #1 - Se espera complejidad: SIMPLE

👤 Pregunta: ¿A qué hora abren?

🔍 [CLASSIFIER] Analizando: '¿A qué hora abren?'
   Complejidad detectada: SIMPLE

🔀 [ROUTER] Decisión: 'SIMPLE' → Modelo Simple (GPT-3.5)

🟢 [SIMPLE MODEL] Procesando con GPT-3.5...
   ✅ Respuesta generada (71 chars)

────────────────────────────────────────────────────────────────────────────────
📊 RESULTADO:
────────────────────────────────────────────────────────────────────────────────

🤖 Modelo usado: gpt-3.5-turbo
📏 Complejidad detectada: SIMPLE
⚠️ Requiere validación humana: No

💬 Respuesta:

Nuestro horario de apertura es de lunes a viernes de 8:00 am a 5:00 pm.

PRUEBA #2 - Se espera complejidad: COMPLEJO

👤 Pregunta: ¿Qué equipo de demolición me recomiendan para un edificio de 5 pisos con concreto armado?

🔍 [CLASSIFIER] Analizando: '¿Qué equipo de demolición me recomiendan para un edificio de 5 pisos con concreto armado?'
   Complejidad detectada: COMPLEJO

🔀 [R

---
# 🐳 PARTE 2: Integración con Ollama (Modelos Locales)

## ¿Por qué Ollama?

**Ollama** permite correr modelos de IA **localmente**, sin costo de API:

### Ventajas:
- ✅ **Costo $0** (no pagas por tokens)
- ✅ **Privacidad total** (datos no salen de tu servidor)
- ✅ **Offline** (funciona sin internet)
- ✅ **Baja latencia** (si tienes GPU local)

### Desventajas:
- ❌ Requiere GPU (o es muy lento en CPU)
- ❌ Modelos open-source son menos potentes que GPT-4
- ❌ Consume recursos de tu servidor

### Caso de uso ideal:
```
Clasificación inicial → Ollama (Llama 3) - GRATIS
                          │
        ¿Requiere razonamiento avanzado?
                 │              │
               No              Sí
                 │              │
           Ollama (gratis)   GPT-4 (pago)
```

**Ahorro:** Usar Ollama para 80% de consultas simples → Reducir costos de API en 80%

## 2.1 Setup de Ollama con Docker

Primero necesitas tener Ollama corriendo. Aquí están las instrucciones:

In [16]:
# Este código verifica si Ollama está corriendo
import requests

def verificar_ollama():
    try:
        response = requests.get("http://localhost:11434/api/tags", timeout=2)
        if response.status_code == 200:
            modelos = response.json().get('models', [])
            print("✅ Ollama está corriendo")
            print(f"\n📦 Modelos disponibles: {len(modelos)}")
            for modelo in modelos:
                print(f"   - {modelo['name']}")
            return True
        else:
            print("⚠️ Ollama responde pero con error")
            return False
    except Exception as e:
        print("❌ Ollama NO está corriendo")
        print("\n📋 Para iniciar Ollama, ejecuta:")
        print("\n   OPCIÓN 1 - Con Docker (recomendado):")
        print("   docker run -d -p 11434:11434 --name ollama ollama/ollama")
        print("   docker exec -it ollama ollama pull llama3.2")
        print("\n   OPCIÓN 2 - Instalación nativa:")
        print("   Descarga desde https://ollama.com")
        print("   ollama pull llama3.2")
        return False

ollama_disponible = verificar_ollama()

✅ Ollama está corriendo

📦 Modelos disponibles: 1
   - llama3.2:latest


## 2.2 Configurar LangChain con Ollama

In [17]:
from langchain_ollama import ChatOllama

# Solo configurar si Ollama está disponible
if ollama_disponible:
    # Modelo local con Ollama
    llm_local = ChatOllama(
        model="llama3.2",  # o "mistral", "codellama", etc.
        base_url="http://localhost:11434",
        temperature=0.3
    )
    
    print("✅ Ollama configurado")
    print("\n🧪 Prueba rápida:")
    
    # Test simple
    response = llm_local.invoke([HumanMessage(content="Di 'Hola, funciono correctamente' en español")])
    print(f"   Respuesta: {response.content}")
    
    print("\n💡 Ahora puedes usar llm_local como cualquier otro LLM de LangChain")
else:
    print("⚠️ Saltando configuración de Ollama")
    print("   Puedes continuar con los modelos de OpenAI")
    llm_local = None

✅ Ollama configurado

🧪 Prueba rápida:
   Respuesta: "Hola, funciona correctamente" es una frase que se puede traducir al inglés como "Hello, it works correctly".

💡 Ahora puedes usar llm_local como cualquier otro LLM de LangChain


## 2.3 Grafo Híbrido: Ollama + OpenAI

El mejor de ambos mundos:
- **Ollama** para tareas simples (gratis, rápido)
- **OpenAI** para tareas complejas (calidad premium)

In [18]:
if ollama_disponible:
    
    class StateHibrido(TypedDict):
        pregunta: str
        es_simple: bool
        respuesta: str
        modelo_usado: str
        costo_estimado: float  # En USD
    
    # Nodo 1: Clasificar con Ollama (gratis)
    def clasificar_con_ollama(state: StateHibrido) -> StateHibrido:
        print("\n🦙 [OLLAMA] Clasificando pregunta (GRATIS)...")
        
        prompt = f"""Clasifica esta pregunta como SIMPLE o COMPLEJA.
SIMPLE: Preguntas sobre horarios, ubicación, contacto, stock básico.
COMPLEJA: Análisis técnico, comparaciones, recomendaciones, cálculos.

Pregunta: {state['pregunta']}

Responde SOLO: SIMPLE o COMPLEJA"""
        
        response = llm_local.invoke([HumanMessage(content=prompt)])
        es_simple = "SIMPLE" in response.content.upper()
        
        print(f"   Clasificación: {'SIMPLE' if es_simple else 'COMPLEJA'}")
        
        return {**state, "es_simple": es_simple, "costo_estimado": 0.0}
    
    # Nodo 2: Responder con Ollama
    def responder_con_ollama(state: StateHibrido) -> StateHibrido:
        print("\n🦙 [OLLAMA] Generando respuesta (GRATIS)...")
        
        prompt = f"""Eres un asistente de CONCESA. Responde brevemente.

Pregunta: {state['pregunta']}

Respuesta:"""
        
        response = llm_local.invoke([HumanMessage(content=prompt)])
        
        print("   ✅ Respuesta generada")
        
        return {
            **state,
            "respuesta": response.content,
            "modelo_usado": "Llama 3.2 (local)",
            "costo_estimado": 0.0
        }
    
    # Nodo 3: Responder con GPT-4o-mini
    def responder_con_openai(state: StateHibrido) -> StateHibrido:
        print("\n🤖 [OPENAI] Generando respuesta con GPT-4o-mini...")
        
        prompt = f"""Eres un experto técnico de CONCESA. Proporciona análisis detallado.

Pregunta: {state['pregunta']}

Respuesta:"""
        
        response = llm_advanced.invoke([HumanMessage(content=prompt)])
        
        # Estimar costo (rough)
        tokens_input = len(prompt) // 4
        tokens_output = len(response.content) // 4
        costo = (tokens_input * 0.15 / 1_000_000) + (tokens_output * 0.60 / 1_000_000)
        
        print(f"   ✅ Respuesta generada (costo estimado: ${costo:.6f})")
        
        return {
            **state,
            "respuesta": response.content,
            "modelo_usado": "GPT-4o-mini (API)",
            "costo_estimado": costo
        }
    
    # Routing
    def decidir_modelo_hibrido(state: StateHibrido) -> Literal["ollama", "openai"]:
        if state["es_simple"]:
            print("\n🔀 [ROUTER] → Ollama (gratis)")
            return "ollama"
        else:
            print("\n🔀 [ROUTER] → OpenAI (calidad premium)")
            return "openai"
    
    # Construir grafo híbrido
    workflow_hibrido = StateGraph(StateHibrido)
    
    workflow_hibrido.add_node("clasificar", clasificar_con_ollama)
    workflow_hibrido.add_node("ollama", responder_con_ollama)
    workflow_hibrido.add_node("openai", responder_con_openai)
    
    workflow_hibrido.add_edge(START, "clasificar")
    workflow_hibrido.add_conditional_edges(
        "clasificar",
        decidir_modelo_hibrido,
        {"ollama": "ollama", "openai": "openai"}
    )
    workflow_hibrido.add_edge("ollama", END)
    workflow_hibrido.add_edge("openai", END)
    
    app_hibrido = workflow_hibrido.compile()
    
    print("\n✅ Grafo Híbrido (Ollama + OpenAI) creado")
    print("\n💰 Estrategia de ahorro:")
    print("   - Preguntas simples → Ollama ($0)")
    print("   - Preguntas complejas → GPT-4o-mini ($$$)")
    print("   - Ahorro potencial: 70-90% en costos de API")

else:
    print("⚠️ Ollama no disponible, saltando configuración híbrida")
    app_hibrido = None


✅ Grafo Híbrido (Ollama + OpenAI) creado

💰 Estrategia de ahorro:
   - Preguntas simples → Ollama ($0)
   - Preguntas complejas → GPT-4o-mini ($$$)
   - Ahorro potencial: 70-90% en costos de API


## 2.4 🎬 DEMO: Sistema Híbrido

In [19]:
if app_hibrido:
    print("="*80)
    print("🎬 DEMO: SISTEMA HÍBRIDO (Ollama + OpenAI)")
    print("="*80)
    
    preguntas_hibrido = [
        "¿Tienen rotomartillos disponibles?",  # SIMPLE → Ollama
        "Necesito comparar las especificaciones técnicas de sus demoledores más potentes para un proyecto de demolición de un edificio de 8 pisos"  # COMPLEJA → OpenAI
    ]
    
    costo_total = 0.0
    
    for i, pregunta in enumerate(preguntas_hibrido, 1):
        print(f"\n{'='*80}")
        print(f"CONSULTA #{i}")
        print(f"{'='*80}")
        print(f"\n👤 Pregunta: {pregunta}")
        
        resultado = app_hibrido.invoke({
            "pregunta": pregunta,
            "es_simple": False,
            "respuesta": "",
            "modelo_usado": "",
            "costo_estimado": 0.0
        })
        
        print(f"\n{'─'*80}")
        print("📊 RESULTADO:")
        print(f"{'─'*80}")
        print(f"\n🤖 Modelo: {resultado['modelo_usado']}")
        print(f"💰 Costo: ${resultado['costo_estimado']:.6f}")
        print(f"\n💬 Respuesta:\n")
        print(resultado['respuesta'])
        
        costo_total += resultado['costo_estimado']
    
    print(f"\n\n{'='*80}")
    print("💰 ANÁLISIS DE COSTOS")
    print("="*80)
    print(f"\nCosto total de las {len(preguntas_hibrido)} consultas: ${costo_total:.6f}")
    print(f"\n🔍 Comparación si usáramos SOLO GPT-4o para todo:")
    print(f"   Costo estimado: ~${costo_total * 3:.6f} (3x más caro)")
    print(f"\n✅ Ahorro con sistema híbrido: ~{((1 - costo_total/(costo_total*3)) * 100):.0f}%")

else:
    print("⚠️ Demo híbrida omitida (Ollama no disponible)")
    print("\nPuedes instalar Ollama para ver esta funcionalidad:")
    print("https://ollama.com")

🎬 DEMO: SISTEMA HÍBRIDO (Ollama + OpenAI)

CONSULTA #1

👤 Pregunta: ¿Tienen rotomartillos disponibles?

🦙 [OLLAMA] Clasificando pregunta (GRATIS)...
   Clasificación: SIMPLE

🔀 [ROUTER] → Ollama (gratis)

🦙 [OLLAMA] Generando respuesta (GRATIS)...
   ✅ Respuesta generada

────────────────────────────────────────────────────────────────────────────────
📊 RESULTADO:
────────────────────────────────────────────────────────────────────────────────

🤖 Modelo: Llama 3.2 (local)
💰 Costo: $0.000000

💬 Respuesta:

Sí, tenemos rotomartillos disponibles en nuestras tiendas y en nuestro sitio web. ¿Necesitas ayuda para encontrar uno?

CONSULTA #2

👤 Pregunta: Necesito comparar las especificaciones técnicas de sus demoledores más potentes para un proyecto de demolición de un edificio de 8 pisos

🦙 [OLLAMA] Clasificando pregunta (GRATIS)...
   Clasificación: COMPLEJA

🔀 [ROUTER] → OpenAI (calidad premium)

🤖 [OPENAI] Generando respuesta con GPT-4o-mini...
   ✅ Respuesta generada (costo estimado: $0.

---
# 📚 PARTE 3: Resumen y Mejores Prácticas

## ✅ Lo que aprendiste:

### 1. Multi-Modelo Strategy
- ✅ Usar diferentes modelos para diferentes tareas
- ✅ Clasificación automática de complejidad
- ✅ Routing inteligente entre modelos
- ✅ Optimización de costos

### 2. Ollama Integration
- ✅ Configurar modelos locales con Docker
- ✅ Integrar Ollama con LangChain
- ✅ Sistema híbrido (local + cloud)
- ✅ Reducir costos con modelos gratis

## 💡 Mejores Prácticas:

### Cuándo usar cada modelo:

```
┌─────────────────┬──────────────┬─────────────────────────┐
│ Tarea           │ Modelo       │ Razón                   │
├─────────────────┼──────────────┼─────────────────────────┤
│ Clasificación   │ GPT-3.5      │ Rápido, barato          │
│ FAQ simple      │ Ollama/3.5   │ Costo $0 o mínimo       │
│ Análisis        │ GPT-4o-mini  │ Balance costo/calidad   │
│ Crítico         │ GPT-4o       │ Máxima calidad          │
│ Código          │ GPT-4o       │ Precisión necesaria     │
│ Embeddings      │ text-3-small │ Más barato para RAG     │
└─────────────────┴──────────────┴─────────────────────────┘
```

### Estrategia de ahorro:

1. **80/20 Rule**: 80% de consultas son simples → Usa modelos baratos
2. **Clasificación primero**: Invierte poco en clasificar, ahorra mucho después
3. **Validación humana**: Casos críticos siempre van a revisión
4. **Caché respuestas**: Para FAQs comunes
5. **Ollama para dev**: Desarrollo local sin costo

## 🚀 Próximos pasos:

En la siguiente sección veremos:
1. **Persistencia con checkpoints** (guardar progreso del grafo)
2. **Human-in-the-loop avanzado** (aprobaciones multi-nivel)
3. **Streaming** (respuestas en tiempo real)
4. **Monitoreo y observability** (LangSmith)

---

# 🎯 Ejercicios Propuestos

## Ejercicio 1: Agregar un cuarto nivel
Modifica el grafo multi-modelo para agregar un nivel "ULTRA-CRÍTICO" que:
- Use GPT-4o
- Requiera validación de 2 humanos
- Genere un reporte PDF

## Ejercicio 2: Implementar caché
Agrega un nodo de "cache" que:
- Verifique si la pregunta ya fue respondida antes
- Si existe, retorne la respuesta cacheada (sin usar API)
- Si no existe, proceda al flujo normal

## Ejercicio 3: Modelo de embeddings local
Integra un modelo de embeddings local con Ollama para:
- Búsqueda semántica sin costo
- Comparar resultados vs OpenAI embeddings
- Medir diferencia en calidad

---

**¡Continúa a la siguiente parte para ver Docker Compose y configuración de producción!**